<a href="https://colab.research.google.com/github/tipiorgup/Tutorial_IDP_SAPs/blob/main/Simulation%20code/Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os
import sys
import shutil
import nglview
# try:
#     if os.path.exists('Tutorial_IDP_SAPs'):
#         print("Found existing directory, attempting to remove...")
#         shutil.rmtree('Tutorial_IDP_SAPs', ignore_errors=True)
#         print("Directory removed successfully")
#     else:
#         print("Directory doesn't exist")
# except Exception as e:
#     print(f"Error while trying to remove directory: {e}")

import ipywidgets as widgets
from IPython.display import display


In [16]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
!git clone https://github.com/tipiorgup/Tutorial_IDP_SAPs.git
!pip install openmm
!pip install MDAnalysis
!pip install ipywidgets
!pip install nglview

Cloning into 'Tutorial_IDP_SAPs'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 123 (delta 42), reused 93 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (123/123), 80.35 KiB | 5.74 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [17]:
os.chdir('Tutorial_IDP_SAPs/Simulation code')

FileNotFoundError: [Errno 2] No such file or directory: 'Tutorial_IDP_SAPs/Simulation code'

In [23]:
global stored_values
stored_values = {
    'Simulation type': None,
    'Sequence': None,
    'Sequence 1': None,
    'Sequence 2': None,
    'Ratio': None,
    'Processed_Ratio': None,
    'peptide_pairs': None
}

# Create the checkbox widgets
checkbox_a = widgets.Checkbox(
    value=False,
    description='Single peptide',
    disabled=False
)

checkbox_b = widgets.Checkbox(
    value=False,
    description='Co-assembly',
    disabled=False
)

# Create text input for option A
text_input_a = widgets.Text(
    description='Sequence:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence (up to 100 characters)'
)

# Create text inputs for option B
text_input_b1 = widgets.Text(
    description='Sequence 1:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence 1 (up to 100 characters)'
)

text_input_b2 = widgets.Text(
    description='Sequence 2:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter sequence 2 (up to 100 characters)'
)

text_input_b3 = widgets.Text(
    description='Ratio:',
    disabled=False,
    layout={'visibility': 'hidden', 'width': '800px'},
    placeholder='Enter ratio'
)

def process_ratio(ratio_str):
    try:
        numbers = ratio_str.replace(' ', '').split('-')
        if len(numbers) == 2:
            num1 = int(numbers[0]) * 100
            num2 = int(numbers[1]) * 100
            return f"{num1}-{num2}"
    except:
        return ratio_str
    return ratio_str

def update_stored_values():
    if checkbox_a.value:
        stored_values['Simulation type'] = 'Single peptide'
        stored_values['Sequence'] = text_input_a.value
        stored_values['Sequence 1'] = text_input_a.value
        stored_values['Sequence 2'] = 'A'  # Constant A as Sequence 2
        stored_values['Ratio'] = '10-0'  # Default ratio
        stored_values['Processed_Ratio'] = process_ratio('10-0')  # Process default ratio
        if text_input_a.value:  # Only create if sequence exists
            stored_values['peptide_pairs'] = f"{text_input_a.value}-A"
    elif checkbox_b.value:
        stored_values['Simulation type'] = 'Co-assembly'
        stored_values['Sequence'] = None
        stored_values['Sequence 1'] = text_input_b1.value
        stored_values['Sequence 2'] = text_input_b2.value
        stored_values['Ratio'] = text_input_b3.value
        stored_values['Processed_Ratio'] = process_ratio(text_input_b3.value)
        if text_input_b1.value and text_input_b2.value:
            stored_values['peptide_pairs'] = f"{text_input_b1.value}-{text_input_b2.value}"

def on_checkbox_change(change):
    if change['owner'] == checkbox_a and change['new']:
        checkbox_b.value = False
        text_input_a.layout.visibility = 'visible'
        text_input_b1.layout.visibility = 'hidden'
        text_input_b2.layout.visibility = 'hidden'
        text_input_b3.layout.visibility = 'hidden'
    elif change['owner'] == checkbox_b and change['new']:
        checkbox_a.value = False
        text_input_a.layout.visibility = 'hidden'
        text_input_b1.layout.visibility = 'visible'
        text_input_b2.layout.visibility = 'visible'
        text_input_b3.layout.visibility = 'visible'

    update_stored_values()

def on_text_change(change):
    update_stored_values()

# Register callbacks
checkbox_a.observe(on_checkbox_change, names='value')
checkbox_b.observe(on_checkbox_change, names='value')
text_input_a.observe(on_text_change, names='value')
text_input_b1.observe(on_text_change, names='value')
text_input_b2.observe(on_text_change, names='value')
text_input_b3.observe(on_text_change, names='value')



In [24]:
widget = widgets.VBox([
    checkbox_a,
    checkbox_b,
    text_input_a,
    text_input_b1,
    text_input_b2,
    text_input_b3
])

In [25]:
display(widget)

In [21]:
stored_values

{'Simulation type': None,
 'Sequence': None,
 'Sequence 1': None,
 'Sequence 2': None,
 'Ratio': None,
 'Processed_Ratio': None,
 'peptide_pairs': None}

In [10]:
#Check in src the file parsers.py, it will show all options available
ionic_strength='0.3'

In [11]:
# First check file
residues_file = 'src/residues.csv'

if not os.path.exists(residues_file):
    print(f"❌ Error: File '{residues_file}' not found!")
else:
    !python clean_pair_code.py --ratio {stored_values['Ratio']} \
        --ionic-strength {ionic_strength} \
        --ph 7.4 \
        --peptide-pair {stored_values['peptide_pairs']} \
        --run-all \
        --residues-file 'src/residues.csv' \
        --steps 1000

Cannot import pdbfixer. This may affect 3SPN2 model.
Processing pair: None
Error processing pair None: invalid literal for int() with base 10: 'None'
Error processing pair None: invalid literal for int() with base 10: 'None'


In [26]:
test_widget = widgets.Text(description='Test:')
display(test_widget)

Text(value='', description='Test:')